In [1]:
from tools.imports import *

In [4]:
import os
import json
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# Function to authenticate and get the service
def authenticate_with_oauth():
    creds = None
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)
    service = build('drive', 'v3', credentials=creds)
    return service

# Function to list files and write to intermediate files
def list_files_and_write(service, output_dir, batch_size=100):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    page_token = None
    batch_num = 0
    file_count = 0
    batch_data = []

    while True:
        response = service.files().list(
            q="'me' in owners",
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType, createdTime, modifiedTime)',
            pageToken=page_token
        ).execute()
        
        items = response.get('files', [])
        if not items:
            break
        
        for item in items:
            batch_data.append(item)
            file_count += 1
            
            if file_count >= batch_size:
                batch_file = os.path.join(output_dir, f'batch_{batch_num}.json')
                with open(batch_file, 'w') as f:
                    json.dump(batch_data, f, indent=4)
                
                batch_num += 1
                file_count = 0
                batch_data = []

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
    
    # Write remaining data if exists
    if batch_data:
        batch_file = os.path.join(output_dir, f'batch_{batch_num}.json')
        with open(batch_file, 'w') as f:
            json.dump(batch_data, f, indent=4)

# Authenticate and get the service
service = authenticate_with_oauth()

# List files and write to intermediate files
output_dir = 'drive_batches'
list_files_and_write(service, output_dir)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59799%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly&state=miQsS6PWdUXORBtyKBqqnoofAFdutT&access_type=offline
